In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LightweightMedicalCNN(nn.Module):
    def __init__(self, num_classes):
        super(LightweightMedicalCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        
        self.res_block1 = self._make_res_block(64, 64)
        self.res_block2 = self._make_res_block(64, 64)  # Additional residual block
        
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        
        self.fc1 = None  # Placeholder for the first fully connected layer
        self.bn5 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, num_classes)
        
    def _make_res_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels)
        )
        
    def _initialize_fc1(self, x):
        # Compute the shape of the tensor after all convolutions and pooling
        size = x.size(1) * x.size(2) * x.size(3)
        self.fc1 = nn.Linear(size, 256).to(x.device)  # Ensure fc1 is on the same device as x
        
    def forward(self, x):
        x = self.pool(self.bn2(F.relu(self.conv2(self.bn1(F.relu(self.conv1(x)))))))
        x = self.dropout(x)
        x = self.pool(self.bn3(F.relu(self.conv3(x))))
        x = self.dropout(x)
        
        x = self.res_block1(x) + x
        x = self.res_block2(F.relu(x)) + x  # Apply second residual block
        x = F.relu(x)
        
        x = self.pool(self.bn4(F.relu(self.conv4(x))))  # Additional convolutional layer after residual blocks
        x = self.dropout(x)
        
        if self.fc1 is None:
            self._initialize_fc1(x)
        
        x = x.view(x.size(0), -1)
        x = self.dropout(self.bn5(F.relu(self.fc1(x))))
        x = self.fc2(x)
        return x

# Example usage
model = LightweightMedicalCNN(num_classes=3)
print(model)


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class ModifiedResNet(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(ModifiedResNet, self).__init__()
        # Charger le modèle ResNet pré-entraîné
        self.model = models.resnet18(pretrained=pretrained)
        
        # Modifier la première couche pour accepter des images en niveaux de gris (1 canal)
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        # Modifier la tête du réseau pour correspondre au nombre de classes
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)
        
    def forward(self, x):
        x = self.model(x)
        return x

# Exemple d'utilisation
num_classes = 25
model_condition = ModifiedResNet(num_classes=num_classes, pretrained=True)
print(model_condition)


In [ ]:
!pip install torchsummary

In [ ]:
# from torchsummary import summary
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# summary(model_condition.to(device), (1, 320, 320))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

if torch.cuda.device_count() > 1:
    print("Utilisons", torch.cuda.device_count(), "GPUs !")
    model = torch.nn.DataParallel(model)
    model_condition = torch.nn.DataParallel(model_condition)

model = model.to(device)
model_condition = model_condition.to(device)

In [ ]:
from tqdm import tqdm
import wandb

In [ ]:
def train_and_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs=5, patience=3, growth_threshold=0.5):
    #Initialiser Weights & Biases
    wandb.init(project="RSNA 2024 project",
               config={
                   "architecture": "LightweightMedicalCNN",
                   "dataset": "RSNA 2024 dataset",
                   "initial_learning_rate": optimizer.param_groups[0]['lr'],
                   "epochs": num_epochs,
                   "initial_weight_decay": optimizer.param_groups[0]['weight_decay']
               })
    epochs_since_adjustment=0
    history = {
        'train_loss': [],
        'train_accuracy': [],
        'val_loss': [],
        'val_accuracy': []
    }
    
    best_val_loss = float('inf')
    epochs_no_improve = 0
    epochs_since_adjustment = 0  # Nouvelle variable pour suivre le nombre d'epochs depuis la dernière mise à jour des hyperparamètres

    previous_train_accuracy = 0
    previous_val_accuracy = 0

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss_items = []
        correct_train = 0
        total_train = 0
        train_tqdm = tqdm(train_loader, desc=f'Training Epoch {epoch+1}', leave=False)
        for images,condi, labels,name in train_tqdm:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss_items.append(loss.item())
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        avg_train_loss = sum(train_loss_items) / len(train_loss_items)
        train_accuracy = 100 * correct_train / total_train
        history['train_loss'].append(avg_train_loss)
        history['train_accuracy'].append(train_accuracy)

        # Log training metrics to Weights & Biases
        wandb.log({'Loss/Train': avg_train_loss,
                   'Accuracy/Train': train_accuracy,
                   'Learning Rate': optimizer.param_groups[0]['lr'],
                   'Weight Decay':optimizer.param_groups[0]['weight_decay'],
                   'epoch': epoch})
        
        # Validation phase
        model.eval()
        val_loss_items = []
        correct_val = 0
        total_val = 0
        val_tqdm = tqdm(val_loader, desc='Validating', leave=False)
        with torch.no_grad():
            for images,condi, labels,name in val_tqdm:
             
                images, labels = images.to(device), labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss_items.append(loss.item())
                
                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        avg_val_loss = sum(val_loss_items) / len(val_loss_items)
        val_accuracy = 100 * correct_val / total_val
        history['val_loss'].append(avg_val_loss)
        history['val_accuracy'].append(val_accuracy)

        # Log validation metrics to Weights & Biases
        wandb.log({'Loss/Validation': avg_val_loss,
                   'Accuracy/Validation': val_accuracy,
                   'Learning Rate': optimizer.param_groups[0]['lr'],
                   'Weight Decay': optimizer.param_groups[0]['weight_decay'],
                   'epoch': epoch})
        
        print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss}, Training Accuracy: {train_accuracy:.2f}%, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%, Learning Rate: {optimizer.param_groups[0]['lr']}, Weight Decay: {optimizer.param_groups[0]['weight_decay']}")

        # Calculate growth rates
        train_accuracy_growth = train_accuracy - previous_train_accuracy
        val_accuracy_growth = val_accuracy - previous_val_accuracy

        # Detect overfitting and adjust learning rate and weight decay
        epochs_since_adjustment += 1
        if epochs_since_adjustment > 4:  # Check if it's been 4 epochs since last adjustment
            if (avg_val_loss >= best_val_loss and train_accuracy > val_accuracy) or \
               (train_accuracy_growth > (1 + growth_threshold) * val_accuracy_growth):
                for param_group in optimizer.param_groups:
                    param_group['lr'] *= 0.8
                    param_group['weight_decay'] *= 1.5
                print(f"Adjusted learning rate to {optimizer.param_groups[0]['lr']} and weight decay to {optimizer.param_groups[0]['weight_decay']} due to overfitting.")
                epochs_since_adjustment = 0  # Reset epochs_since_adjustment after adjustment



        # Update previous accuracies
        previous_train_accuracy = train_accuracy
        previous_val_accuracy = val_accuracy

        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        
        if epochs_no_improve == patience:
            print(f"Early stopping triggered after {epoch + 1} epochs.")
            break


    wandb.finish()
    return history

In [ ]:
def train_and_validate_condition(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, patience=3, growth_threshold=0.5):
    #Initialiser Weights & Biases
    wandb.init(project="RSNA 2024 project_condition",
               config={
                   "architecture": "LightweightMedicalCNN",
                   "dataset": "RSNA 2024 dataset",
                   "initial_learning_rate": optimizer.param_groups[0]['lr'],
                   "epochs": num_epochs,
                   "initial_weight_decay": optimizer.param_groups[0]['weight_decay']
               })
    epochs_since_adjustment=0
    history = {
        'train_loss': [],
        'train_accuracy': [],
        'val_loss': [],
        'val_accuracy': []
    }
    
    best_val_loss = float('inf')
    epochs_no_improve = 0
    epochs_since_adjustment = 0  # Nouvelle variable pour suivre le nombre d'epochs depuis la dernière mise à jour des hyperparamètres

    previous_train_accuracy = 0
    previous_val_accuracy = 0

    for epoch in range(num_epochs):
        # Training phase
        model_condition.train()
        train_loss_items = []
        correct_train = 0
        total_train = 0
        train_tqdm = tqdm(train_loader, desc=f'Training Epoch {epoch+1}', leave=False)
        for images,condi, labels,name in train_tqdm:
            images, labels = images.to(device), labels.to(device)
     
            optimizer.zero_grad()
            outputs = model_condition(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss_items.append(loss.item())
            _, predicted = torch.max(outputs.data, 1)
            total_train += condi.size(0)
            correct_train += (predicted == labels).sum().item()

        avg_train_loss = sum(train_loss_items) / len(train_loss_items)
        train_accuracy = 100 * correct_train / total_train
        history['train_loss'].append(avg_train_loss)
        history['train_accuracy'].append(train_accuracy)

        # Log training metrics to Weights & Biases
        wandb.log({'Loss/Train': avg_train_loss,
                   'Accuracy/Train': train_accuracy,
                   'Learning Rate': optimizer.param_groups[0]['lr'],
                   'Weight Decay':optimizer.param_groups[0]['weight_decay'],
                   'epoch': epoch})
        
        # Validation phase
        model_condition.eval()
        val_loss_items = []
        correct_val = 0
        total_val = 0
        val_tqdm = tqdm(val_loader, desc='Validating', leave=False)
        with torch.no_grad():
            for images,condi, labels,name in val_tqdm:
             
                images, condi = images.to(device), condi.to(device)
                
                outputs = model_condition(images)
                loss = criterion(outputs, labels)
                val_loss_items.append(loss.item())
                
                _, predicted = torch.max(outputs.data, 1)
                total_val += condi.size(0)
                correct_val += (predicted == labels).sum().item()

        avg_val_loss = sum(val_loss_items) / len(val_loss_items)
        val_accuracy = 100 * correct_val / total_val
        history['val_loss'].append(avg_val_loss)
        history['val_accuracy'].append(val_accuracy)

        # Log validation metrics to Weights & Biases
        wandb.log({'Loss/Validation': avg_val_loss,
                   'Accuracy/Validation': val_accuracy,
                   'Learning Rate': optimizer.param_groups[0]['lr'],
                   'Weight Decay': optimizer.param_groups[0]['weight_decay'],
                   'epoch': epoch})
        
        print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss}, Training Accuracy: {train_accuracy:.2f}%, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%, Learning Rate: {optimizer.param_groups[0]['lr']}, Weight Decay: {optimizer.param_groups[0]['weight_decay']}")

        # Calculate growth rates
        train_accuracy_growth = train_accuracy - previous_train_accuracy
        val_accuracy_growth = val_accuracy - previous_val_accuracy

        # Detect overfitting and adjust learning rate and weight decay
        epochs_since_adjustment += 1
        if epochs_since_adjustment > 4:  # Check if it's been 4 epochs since last adjustment
            if (avg_val_loss >= best_val_loss and train_accuracy > val_accuracy) or \
               (train_accuracy_growth > (1 + growth_threshold) * val_accuracy_growth):
                for param_group in optimizer.param_groups:
                    param_group['lr'] *= 0.8
                    param_group['weight_decay'] *= 1.5
                print(f"Adjusted learning rate to {optimizer.param_groups[0]['lr']} and weight decay to {optimizer.param_groups[0]['weight_decay']} due to overfitting.")
                epochs_since_adjustment = 0  # Reset epochs_since_adjustment after adjustment



        # Update previous accuracies
        previous_train_accuracy = train_accuracy
        previous_val_accuracy = val_accuracy

        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        
        if epochs_no_improve == patience:
            print(f"Early stopping triggered after {epoch + 1} epochs.")
            break


    wandb.finish()
    return history

In [ ]:
#train_loader = DataLoader(train_dataset, batch_size=120, shuffle=True,num_workers=4)
from torch.utils.data import ConcatDataset
train_loader = DataLoader(ConcatDataset([train_dataset]), batch_size=100, shuffle=True,num_workers=4,pin_memory=True)
val_loader = DataLoader(valid_dataset, batch_size=100, shuffle=False,num_workers=4,pin_memory=True)

In [ ]:
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
optimizer=torch.optim.AdamW(model_condition.parameters(), lr=1e-5, weight_decay=1e-4) #meilleur config jusqu'a present


In [ ]:
history_condition=train_and_validate_condition(model_condition, train_loader, val_loader, criterion, optimizer)

In [ ]:
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
optimizer=torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4) #meilleur config jusqu'a present


In [ ]:
history=train_and_validate(model, train_loader, val_loader, criterion, optimizer)

In [ ]:
#torch.save(model.state_dict(), '/kaggle/working/model_weights_Resnet50_1.pth')

In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history, title="Training and Validation"):
    """
    Trace les courbes de perte et de précision pour les ensembles d'entraînement et de validation en utilisant l'objet history.

    Args:
    history: Un dictionnaire contenant les clés 'loss', 'val_loss', 'accuracy', et 'val_accuracy',
             qui mappent à des listes des valeurs correspondantes à travers les époques.
    title (str): Titre du graphique.
    """
    train_loss = history['train_loss']
    val_loss = history['val_loss']
    train_acc = history['train_accuracy']
    val_acc = history['val_accuracy']
    epochs = range(1, len(train_loss) + 1)

    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, 'bo-', label='Training Loss')
    plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
    plt.title(title + ' - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_acc, 'bo-', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'ro-', label='Validation Accuracy')
    plt.title(title + ' - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

In [ ]:
plot_training_history(history_condition)

In [ ]:
plot_training_history(history)